In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [ ]:
import acquire

In [ ]:
df = acquire.get_data()

In [ ]:
df.info()

In [ ]:
df = acquire.clean_data(df)

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(df2)

In [ ]:
df3 = scaler.transform(df2)

In [ ]:
columns = list(df2.columns)

In [ ]:
columns

In [ ]:
df_scaled = pd.DataFrame(df3, columns=columns)

In [ ]:
df_scaled2 = df_scaled.set_index(df2.index)

In [ ]:
df2.Floors.plot(figsize=(15, 6))

In [ ]:
floors = df2.Floors

In [ ]:
floors